In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 시드고정

In [2]:
import random
import os
import numpy as np
import torch
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(41) # Seed 고정

## Import

In [3]:
import pandas as pd
import numpy as np

from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
train_df = pd.read_csv('./drive/MyDrive/신용카드 사기 데이콘/open/train.csv')
val_df = pd.read_csv('./drive/MyDrive/신용카드 사기 데이콘/open/val.csv')
test = pd.read_csv('./drive/MyDrive/신용카드 사기 데이콘/open/test.csv')

print(train_df.shape, val_df.shape, test.shape)

(113842, 31) (28462, 32) (142503, 31)


In [5]:
train_df

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,3,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972
1,4,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972
2,6,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,-0.256131,-0.994960
3,8,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0.262698,-0.994901
4,9,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.994900,-0.994901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113837,284796,-12.516732,10.187818,-8.476671,-2.510473,-4.586669,-1.394465,-3.632516,5.498583,4.893089,...,-0.944759,-1.565026,0.890675,-1.253276,1.786717,0.320763,2.090712,1.232864,-0.169496,1.034857
113838,284797,1.884849,-0.143540,-0.999943,1.506772,-0.035300,-0.613638,0.190241,-0.249058,0.666458,...,0.144008,0.634646,-0.042114,-0.053206,0.316403,-0.461441,0.018265,-0.041068,0.530986,1.034881
113839,284798,-0.241923,0.712247,0.399806,-0.463406,0.244531,-1.343668,0.929369,-0.206210,0.106234,...,-0.228876,-0.514376,0.279598,0.371441,-0.559238,0.113144,0.131507,0.081265,-0.230699,1.034904
113840,284802,0.120316,0.931005,-0.546012,-0.745097,1.130314,-0.235973,0.812722,0.115093,-0.204064,...,-0.314205,-0.808520,0.050343,0.102800,-0.435870,0.124079,0.217940,0.068803,-0.269825,1.034939


In [6]:
train_df = train_df.drop(train_df.columns[1:], axis=1)
train_df

,ID
0,3
1,4
2,6
3,8
4,9
...,...
113837,284796
113838,284797
113839,284798
113840,284802


In [7]:
test = test.drop(test.columns[1:], axis=1)
test

,ID
0,AAAA0x1
1,AAAA0x2
2,AAAA0x5
3,AAAA0x7
4,AAAA0xc
...,...
142498,0x4587f
142499,0x45880
142500,0x45884
142501,0x45885


In [8]:
val_df = val_df.drop(val_df.columns[1:-1], axis=1)
val_df

,ID,Class
0,10,0
1,22,0
2,63,0
3,69,0
4,83,0
...,...,...
28457,284769,0
28458,284779,0
28459,284790,0
28460,284801,0


In [9]:
train_tsne = pd.read_csv('./drive/MyDrive/신용카드 사기 데이콘/sohyeon/t-sne files for train,test,valid/tsne_train.csv')
val_tsne = pd.read_csv('./drive/MyDrive/신용카드 사기 데이콘/sohyeon/t-sne files for train,test,valid/tsne_valid.csv')
test_tsne = pd.read_csv('./drive/MyDrive/신용카드 사기 데이콘/sohyeon/t-sne files for train,test,valid/tsne_test.csv')

print(train_tsne.shape, val_tsne.shape, test_tsne.shape)

(113842, 3) (28462, 3) (142503, 3)


In [10]:
train_tsne = train_tsne.drop(train_tsne.columns[0], axis=1)
val_tsne = val_tsne.drop(val_tsne.columns[0], axis=1)
test_tsne = test_tsne.drop(test_tsne.columns[0], axis=1)

In [11]:
train_tsne

,new_V1,new_V2
0,-9.116989,7.851337
1,-7.688793,13.343833
2,-2.588786,-32.391903
3,13.190283,8.997262
4,2.803079,27.292868
...,...,...
113837,-8.377257,23.327465
113838,-33.948685,-26.659150
113839,5.462154,-43.065815
113840,20.516718,-24.394896


In [12]:
merged_train = pd.concat([train_df, train_tsne],axis=1)
merged_train

,ID,new_V1,new_V2
0,3,-9.116989,7.851337
1,4,-7.688793,13.343833
2,6,-2.588786,-32.391903
3,8,13.190283,8.997262
4,9,2.803079,27.292868
...,...,...,...
113837,284796,-8.377257,23.327465
113838,284797,-33.948685,-26.659150
113839,284798,5.462154,-43.065815
113840,284802,20.516718,-24.394896


In [13]:
merged_val = pd.concat([val_df, val_tsne],axis=1)
merged_val

,ID,Class,new_V1,new_V2
0,10,0,2.707155,-62.847298
1,22,0,15.791712,-14.021538
2,63,0,-9.664258,-5.983323
3,69,0,13.704683,-15.461807
4,83,0,-22.406229,6.664095
...,...,...,...,...
28457,284769,0,-31.941423,21.972034
28458,284779,0,2.866337,50.081707
28459,284790,0,-23.444632,-0.471761
28460,284801,0,2.885701,-11.147281


In [14]:
merged_val = merged_val[['ID', 'new_V1', 'new_V2','Class']]
merged_val


,ID,new_V1,new_V2,Class
0,10,2.707155,-62.847298,0
1,22,15.791712,-14.021538,0
2,63,-9.664258,-5.983323,0
3,69,13.704683,-15.461807,0
4,83,-22.406229,6.664095,0
...,...,...,...,...
28457,284769,-31.941423,21.972034,0
28458,284779,2.866337,50.081707,0
28459,284790,-23.444632,-0.471761,0
28460,284801,2.885701,-11.147281,0


In [15]:
merged_test = pd.concat([test, test_tsne],axis=1)
merged_test

,ID,new_V1,new_V2
0,AAAA0x1,-0.323882,19.384823
1,AAAA0x2,-0.445384,-36.328590
2,AAAA0x5,1.251372,20.719603
3,AAAA0x7,-27.450216,8.519989
4,AAAA0xc,-28.447172,5.382051
...,...,...,...
142498,0x4587f,21.841896,-3.890553
142499,0x45880,-11.982875,22.497680
142500,0x45884,23.280870,13.313768
142501,0x45885,-25.360207,-21.347479


In [16]:
from sklearn import svm
from sklearn.datasets import make_moons, make_blobs
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import make_pipeline


In [17]:
# Train dataset은 Label이 존재하지 않음
train_x = merged_train.drop(columns=['ID']) # Input Data

In [24]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(train_x)

# 데이터 변환
mMscaled_data = mMscaler.transform(train_x)

# 데이터 프레임으로 저장
mMscaled_data = pd.DataFrame(mMscaled_data)

In [26]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

## model define, fit

In [27]:
model = EllipticEnvelope(support_fraction = 0.994, contamination = 0.00112, random_state = 42)
model.fit(mMscaled_data)
val_x = merged_val.drop(columns=['ID', 'Class']) # Input Data
val_y = merged_val['Class'] # Label

val_pred = model.predict(val_x) # model prediction
val_pred = get_pred_label(val_pred)
val_score = f1_score(val_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y, val_pred))

Validation F1 Score : [0.0010529271374420891]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     28432
           1       0.00      1.00      0.00        30

    accuracy                           0.00     28462
   macro avg       0.00      0.50      0.00     28462
weighted avg       0.00      0.00      0.00     28462



## test

In [28]:
test_x = merged_test.drop(columns=['ID'])

In [29]:
#best model : model "EllipticEnvelope"
test_pred = model.predict(test_x) # model prediction
test_pred = get_pred_label(test_pred)

## submission

In [30]:
submit = pd.read_csv('./drive/MyDrive/신용카드 사기 데이콘/open/sample_submission.csv')
submit.head()

,ID,Class
0,AAAA0x1,1
1,AAAA0x2,1
2,AAAA0x5,1
3,AAAA0x7,1
4,AAAA0xc,1


In [31]:
submit['Class'] = test_pred
submit.to_csv('./drive/MyDrive/신용카드 사기 데이콘/open/elliptic_tsne_minmaxscaling_submit.csv', index=False)